In [1]:
import os
import sqlite3

id2tissue = [t.split('.')[0][3:] for t in os.listdir('../../../data/predix_db/GTEx-V6p-HapMap-2016-09-08/') if t[-3:] == '.gz']

In [2]:
import pandas as pd
import pickle
import numpy as np
import gzip

for t in id2tissue:
    t0 = t.replace('-', '_')
    df = pd.read_csv('../../../data/predix_db/GTEx-V6p-HapMap-2016-09-08/TW_%s.txt.gz' % t, sep = ' ')
    model_genes = set(pd.read_csv('../../processed_data/model_genes/%s-model_genes.txt' % t0, header = None)[0])
    
    
    with sqlite3.connect('../../../data/predix_db/GTEx-V6p-HapMap-2016-09-08/TW_%s_0.5.db' % t) as conn:
        c = conn.cursor()

        gene2var = {}

        for g, dfg in df.groupby('GENE'):
            cmd = (g,)
            c.execute('SELECT * FROM weights WHERE gene=?', cmd)
            id2rsid = sorted(set(dfg.RSID1))

            res = [item for item in c.fetchall()]

            if not res:
                continue

            assert(set(id2rsid) == set([item[0] for item in res]))

            rsid2id = {r:i for i, r in enumerate(id2rsid)}
            mat = np.zeros((len(id2rsid), len(id2rsid)))
            for r1, r2, v in zip(dfg.RSID1, dfg.RSID2, dfg.VALUE):
                mat[rsid2id[r1], rsid2id[r2]] = v
                mat[rsid2id[r2], rsid2id[r1]] = v

            vec = np.zeros(len(id2rsid))
            for r, g, v, _, _ in res:
                vec[rsid2id[r]] = v

            gene2var[g] = vec.dot(mat).dot(vec)

        with gzip.open('../../processed_data/model_genes/var/%s-gene2var.csv.gz' % t0, 'wb') as writer:
            for g, v in gene2var.items():
                writer.write('%s,%f\n' % (g, v))
